# STOCKS

In [20]:
import pandas as pd 
# import numpy as np
from pandas_datareader import data as pdr
from functools import reduce
# import time
# from datetime import datetime
# #from momentum import *
# from portfolio_funcs import get_weights, simulate_portfolios, factible_weights
# from trade_utils import *


import momentum as mm
from ta import trend, momentum
from investment import Portfolio
from trade_utils import create_rollings, create_shifts
from inspect import getmembers, isfunction

## Download Tickers

In [48]:
# Download Tickers
file = "/home/derek/Documentos/Yahoo-ticker-symbol-downloader/generic.pickle"

tickers = pd.read_pickle(file)


from multiprocessing import Pool

def append(i):
#     lista.append(i.getRow())
    return i.getRow()

p = Pool(7)

eq = p.map(append,tickers.symbols.values())

df = pd.DataFrame(eq, columns = tickers.getRowHeader())

df.head()

df.Type.value_counts()

df.TypeDisplay.value_counts().index

df = df[df.TypeDisplay.isin(['Equity', 'Index', 'Fund', 'ETF'])]

df.to_pickle("tickers.pkl")

## Download Data

In [21]:
tickers = pd.read_pickle("tickers.pkl")

In [22]:
# budget, tickers
start='2000-01-01'
end=None
interval="d"
tickers_change=[]
currency_change=["MXN=X"]

def get_data(tick,start='2000-01-01',end=None,interval="d"):

    data = pdr.get_data_yahoo(tick, 
                start=start, 
                end=end, 
                interval=interval) 
#     data = merge_list(data_list)

    data = data.groupby(by='Date').mean()     
#     adj_close = data['Adj Close'].fillna(method = 'ffill').dropna()                         
    return data

In [23]:
def merge_list(data_list):
        data = reduce(lambda  left,right: pd.merge(left,right,
                                                        left_index=True,
                                                        right_index=True,
                                                        how='outer'),                                                         
                                                        data_list)
        return data
def remove_unary(work_df,threshold=0.85):
    f = pd.DataFrame(columns=['col','first'])
    for c in work_df:
        f = f.append({'col':c,'first': work_df[c].value_counts(True,dropna=False).values[0]},ignore_index=True)

    cols = f[f["first"]>=threshold]['col']
#     cols = [x for x in list(cols) if x not in ('fri', 'mon', 'thu', 'tue', 'wed')]
    return list(cols)

In [24]:

import momentum as mm
from inspect import getmembers, isfunction


In [25]:
def pipeline(t):
    try:
        ticker = get_data(t,start=start,end=end,interval=interval)
        if len(ticker)<180:

            return None
        high,low,close,volume = ticker["High"].dropna(), ticker["Low"].dropna(), ticker["Adj Close"].dropna(), ticker["Volume"].dropna() 
        classes = [
            trend.ADXIndicator(high, low, close).adx(),
            trend.AroonIndicator(close).aroon_indicator(),
            trend.CCIIndicator(high, low, close).cci(),
            trend.DPOIndicator(close,).dpo(),
            trend.EMAIndicator(close,).ema_indicator(),
            trend.IchimokuIndicator(high, low,).ichimoku_base_line(),
            trend.KSTIndicator(close).kst_diff(),
            trend.MACD(close,).macd_diff(),
            trend.MassIndex(high, low,).mass_index(),
            trend.PSARIndicator(high, low, close,).psar(),
            trend.SMAIndicator(close,14).sma_indicator(),
            trend.STCIndicator(close,).stc(),
            trend.TRIXIndicator(close,).trix(),
            trend.VortexIndicator(high, low, close,).vortex_indicator_diff(),
            trend.WMAIndicator(close,).wma(),
            momentum.AwesomeOscillatorIndicator(high, low,).awesome_oscillator(),
            momentum.KAMAIndicator(close,).kama(),
            momentum.PercentagePriceOscillator(close,).ppo(),
            momentum.PercentageVolumeOscillator(volume,).pvo(),
            momentum.ROCIndicator(close,).roc(),
            momentum.RSIIndicator(close,).rsi(),
            momentum.StochRSIIndicator(close,).stochrsi_k(),
            momentum.StochasticOscillator(high, low, close,).stoch(),
            momentum.TSIIndicator(close,).tsi(),
            momentum.UltimateOscillator(high, low, close,).ultimate_oscillator(),
            momentum.WilliamsRIndicator(high, low, close,).williams_r()
        ]

        new_feats = merge_list(classes)#.shift(1)
        c = new_feats.tail(180).isnull().sum()
        c = c[c>180]
        new_feats.drop(columns = c.index, inplace=True)

        new_feats["adj_close"] = ticker["Adj Close"]#.pct_change().dropna()
        new_feats["profits"] = ticker["Adj Close"].pct_change()
        # new_feats["std_30"] = new_feats.adj_close.rolling(30).std()
        new_feats = new_feats.fillna(method="ffill")
        new_feats["weekday"] = new_feats.index.weekday#.replace({0:"mon",1:'tue',2:'wed',3:'thu',4:'fri',5:'sat',6:'sun'})
        new_feats["day"] = new_feats.index.day
        new_feats["month"] = new_feats.index.month
        dic = dict(zip([x[0] for x in(getmembers(mm, isfunction)) if x[0] not in ("D_line","RSI","relu","relu_neg","tail_ratio","ultimate",)],[
                                                    mm.CCI,
                                                    mm.K_line,
                                                    mm.MACD,
                                                    mm.MACD_x,
                                                    mm.RS,
                                                    mm.RoMad,
                                                    mm.awesome_oscilator,
                                                    mm.exp_2_avg,
                                                    mm.exp_e_avg,
                                                    mm.exp_x_avg,
                                                    mm.mdd,
                                                    mm.momentum,
                                                    mm.omega_ratio,
                                                    mm.pond_avg,
                                                    mm.sharpe_ratio,
                                                    mm.sortino_ratio,
                                                    mm.williams,                                            
                                                    ]))

        rollings = create_rollings(work_df=new_feats,
                                   columns=["profits"],
                                   agg_funcs=dict(dic,**{'std':'std'}),
                                   windows=[5,10,20,30],
                                   min_periods=None)

        shifts = create_shifts(work_df=new_feats, columns=['profits'], shifts=range(1,31))
        df = merge_list([new_feats,rollings,shifts])
        df.drop(columns=remove_unary(df)+['rolling_profits_CCI_10', 'rolling_profits_CCI_5'],inplace=True)

        df.to_pickle("/home/derek/Documentos/Hackathon/trading/data/"+t+".pkl")
        print(t)
    except:
        pass
    return None

In [44]:
import os
done = [x[:-4] for x in os.listdir("/home/derek/Documentos/Hackathon/trading/data/")]

In [39]:
from multiprocessing import Pool

pool = Pool(7)

/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in 

RWMHX


/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


GMED.MX


/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


...

/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


0P00000CHJ.F
UNIT


/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


GOODM


/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/derek/venv/trading/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in 

In [45]:
stocks = ([x for x in  tickers.Ticker.sample(frac=1) if x not in done])

KeyboardInterrupt: 

In [ ]:
stocks[0]

In [46]:
len(done),len(tickers.Ticker), len(stocks)

(2195, 204433, 202903)

In [43]:
pool.map(pipeline,stocks)

KeyboardInterrupt: 

In [ ]:
for t in tickers.Ticker:
    pipeline(t)

In [ ]:
df = merge_list([new_feats,rollings,shifts])